# Setup

In [43]:
# Standard-ish set of imports copy-pasted from ARENA notebooks

from nnsight import LanguageModel

import gc
import itertools
import math
import os
import random
import sys
from collections import Counter, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Any, Callable, Literal, TypeAlias
import json

import einops
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import torch as t
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from rich import print as rprint
from rich.table import Table
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from sae_vis import SaeVisConfig, SaeVisData, SaeVisLayoutConfig
from tabulate import tabulate
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer, utils
from transformer_lens.hook_points import HookPoint

device = "cuda" if t.cuda.is_available() else "mps" if t.backends.mps.is_available() else "cpu"

In [44]:
## check memory usage

if t.cuda.is_available():
    gpu_id = 0  # Set to your target GPU ID
    total_memory = t.cuda.get_device_properties(gpu_id).total_memory
    allocated_memory = t.cuda.memory_allocated(gpu_id)
    cached_memory = t.cuda.memory_reserved(gpu_id)

    print(f"Total GPU Memory: {total_memory / 1024**2:.2f} MB")
    print(f"Allocated GPU Memory: {allocated_memory / 1024**2:.2f} MB")
    print(f"Cached GPU Memory: {cached_memory / 1024**2:.2f} MB")
elif t.backends.mps.is_available():
    # MPS (Metal Performance Shaders) for Mac
    print("MPS is available.")
    # Note: As of now, PyTorch doesn't provide direct memory management functions for MPS
    print("Memory information is not available for MPS.")
else:
    print("Neither CUDA nor MPS is available.")

Total GPU Memory: 45589.06 MB
Allocated GPU Memory: 30.89 MB
Cached GPU Memory: 492.00 MB


In [45]:
# Clear out GPU memory to avoid out-of-memory errors
# Re-run this cell whenever memory usage gets high.

gc.collect()
t.cuda.empty_cache()


# Helper functions

In [46]:
# Fairly uninteresting helper functions

def load_tensor(filename):
    if device == "mps":
        tensor = t.load(filename, map_location="cpu")
        tensor.to(device, dtype=t.float32)
    else:
        tensor = t.load(filename, map_location="cpu")
        tensor.to(device, dtype=t.float32)
    return tensor

def get_second_min(x):
    min_value = t.min(x)
    mask = x != min_value
    second_min_value = t.min(x[mask])

    return second_min_value

def clear_memory_after(f):
    def func(*args, **kwargs):
        result = f(*args, **kwargs)
        gc.collect()
        t.cuda.empty_cache()
        return result
    return func

@clear_memory_after
def get_activations(prompts, sae_name, sae_id):
    t.set_grad_enabled(False)
    gemma2: HookedSAETransformer = HookedSAETransformer.from_pretrained("gemma-2-2b-it", device=device)
    gemma2_sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=sae_name,
        sae_id=sae_id,
        device=str(device),
    )

    all_sae_acts_post = []

    for prompt in tqdm(prompts):
        # Get top activations on final token
        _, cache = gemma2.run_with_cache_with_saes(
            prompt,
            saes=[gemma2_sae],
            stop_at_layer=gemma2_sae.cfg.hook_layer + 1,
        )
        sae_acts_post = cache[f"{gemma2_sae.cfg.hook_name}.hook_sae_acts_post"][0, -1, :]
        all_sae_acts_post.append(sae_acts_post)

    return t.stack(all_sae_acts_post)

def get_frac_active(sae_activations):
    return (sae_activations > 0).sum(dim=(0)) / sae_activations.shape[0]

def smooth_frac_active(frac_active):
    # Smooth the vector to avoid zeroes by arbitrarily adding half the second
    # smallest value.  This allows dividing by the vector without errors.
    # This is arbitrary; find a better way...
    return t.where(frac_active == 0, get_second_min(frac_active)/2, frac_active)

def get_relative_activation(unsmoothed_frac_active_interest, unsmoothed_frac_active_baseline):
    # Rough statistic: a latent is "interesting" if its activation is big compared to
    # the activation in some baseline.
    return smooth_frac_active(unsmoothed_frac_active_interest)/smooth_frac_active(unsmoothed_frac_active_baseline)

# Experiments

In [47]:
# Find and display interesting latents, comparing activations in some set of harmful data to some baseline.

# See https://www.neuronpedia.org/api-doc for instructions on how to get a Neuronpedia API key
# Save it in this file.
with open('/workspace/neuronpedia-api', 'r') as f:
    neuronpedia_headers = {"X-Api-Key": f.read().strip()}

# Cache calls to fetch explanations from Neuronpedia to make re-runs quick, and since
# some latents appear for multiple sets of harms.
# Note: these are auto-interpretation, so take with a grain of salt, but they have some value
# for quickly getting a sense of something.
try:
    if EXPLANATION_CACHE:
        print("Cache EXPLANATION_CACHE already exists, not overwriting it to avoid repeated API calls!")
except NameError:
    EXPLANATION_CACHE = {}

def display_interesting_latents(harm_activations, baseline_activations, title="Interesting latents", min_harm_frac_active=0.3, num_latents=20, fetch_explanations_from_neuronpedia=True):
    global EXPLANATION_CACHE

    harm_frac_active = get_frac_active(harm_activations)
    baseline_frac_active = get_frac_active(baseline_activations)
    ratio = get_relative_activation(harm_frac_active, baseline_frac_active)
    table = Table("Latent", "Frac harm", "Frac base", "Ratio", "Explanation",
                title=title, show_lines=True)
    for latent in t.where(harm_frac_active>min_harm_frac_active, ratio, 0).topk(k=num_latents).indices:
        path = f"feature/gemma-2-2b/{layer}-gemmascope-res-16k/{latent}"
        if fetch_explanations_from_neuronpedia:
            if path in EXPLANATION_CACHE:
                explanation = EXPLANATION_CACHE[path]
            else:
                response = requests.get(f"https://www.neuronpedia.org/api/{path}", headers=neuronpedia_headers)
                explanations = response.json().get('explanations', [])
                explanation = explanations[0].get('description', "(unknown)") if explanations else "(unknown)"
                EXPLANATION_CACHE[path] = explanation
        else:
            explanation = "Not fetched"
        table.add_row(f"[link=https://www.neuronpedia.org/{path}]Neuronpedia {latent}[/]",
                    f"{harm_frac_active[latent].item():.4f}", f"{baseline_frac_active[latent].item():.4f}", f"{ratio[latent].item():.2f}",
                    explanation)
    rprint(table)


Cache EXPLANATION_CACHE already exists, not overwriting it to avoid repeated API calls!


In [48]:
layer = 5

sae_name = "gemma-scope-2b-pt-res-canonical"
sae_id = f"layer_{layer}/width_16k/canonical"

sae_act_advbench = load_tensor(f'../data/sae_acts/{sae_name}/{sae_id}_advbench.pt')
sae_act_alpaca = load_tensor(f'../data/sae_acts/{sae_name}/{sae_id}_alpaca_10000.pt')



/tmp/ipykernel_4305/3316404989.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = t.load(filename, map_location="cpu")


In [49]:

display_interesting_latents(sae_act_advbench, sae_act_alpaca, title="Interesting latents more common in advbench than alpaca")

                              Interesting latents more common in advbench than alpaca                              
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=763387;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15484\Neuronpedia 15484]8;;\ │ 0.3288    │ 0.0005    │ 657.69 │ instances of legal terms and phrases related to fraud and  │
│                   │           │           │        │ criminal charges                                           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=797132;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/5181\Neuronpedia 5181]8;;\  │ 0.4538    │ 0.0037    │ 122.66 │ terms related to violence and destruction                  │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=546579;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/5502\Neuronpedia 5502]8;;\  │ 0.4077    │ 0.0043    │ 94.81  │ references to organized crime and illegal activities,      │
│                   │           │           │        │ particularly those involving drug trafficking              │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=832868;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/14030\Neuronpedia 14030]8;;\ │ 0.3385    │ 0.0130    │ 26.04  │ references to various forms of crime and victimization,    │
│                   │           │           │        │ particularly related to violence and fraud                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=297921;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/9988\Neuronpedia 9988]8;;\  │ 0.3462    │ 0.0151    │ 22.92  │ biological processes and their associated proteins or      │
│                   │           │           │        │ structures                                                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=139594;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/3205\Neuronpedia 3205]8;;\  │ 0.6173    │ 0.0289    │ 21.36  │ terms related to specific legal contexts and their         │
│                   │           │           │        │ associated parameters                                      │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=5853;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4776\Neuronpedia 4776]8;;\  │ 0.3346    │ 0.0157    │ 21.31  │  phrases related to geographical descriptions and          │
│                   │           │           │        │ attributes                                                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=489162;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/3182\Neuronpedia 3182]8;;\  │ 0.4635    │ 0.0258    │ 17.96  │ terms and concepts related to legal actions and procedures │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=950147;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/7114\Neuronpedia 7114]8;;\  │ 0.5423    │ 0.0389    │ 13.94  │  phrases related to historical or legal contexts involving │
│            

In [50]:
len(EXPLANATION_CACHE)

94

In [51]:

with open('/workspace/refusal_direction/dataset/processed/strongreject.json', 'r') as file:
    strongreject_raw = json.load(file)

strongreject = defaultdict(list)
for item in strongreject_raw:
  strongreject[item['category']].append(item['instruction'])

strongreject.keys()

dict_keys(['Disinformation and deception', 'Hate, harassment and discrimination', 'Illegal goods and services', 'Non-violent crimes', 'Sexual content', 'Violence'])

In [52]:
strongreject_activations = {}
for category in strongreject:
    print(f"Loading activations for {category}")
    strongreject_activations[category] = get_activations(strongreject[category], "gemma-scope-2b-pt-res-canonical", f"layer_{layer}/width_16k/canonical")

Loading activations for Disinformation and deception


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


Loaded pretrained model gemma-2-2b-it into HookedTransformer


100%|██████████| 50/50 [00:03<00:00, 13.32it/s]


Loading activations for Hate, harassment and discrimination


...

100%|██████████| 50/50 [00:03<00:00, 13.91it/s]


Loading activations for Violence


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]


Loaded pretrained model gemma-2-2b-it into HookedTransformer


100%|██████████| 54/54 [00:04<00:00, 13.28it/s]


In [53]:
for category in strongreject_activations:
  display_interesting_latents(strongreject_activations[category].cpu(), sae_act_alpaca, min_harm_frac_active=0.1, title=f"Interesting latents more common in strongreject's \"{category}\" than alpaca")

           Interesting latents more common in strongreject's "Disinformation and deception" than alpaca            
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=633112;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.6000    │ 0.0009    │ 666.67 │ various sentence structures and punctuation                │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=283011;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/12923\Neuronpedia 12923]8;;\ │ 0.1200    │ 0.0002    │ 600.00 │ phrases discussing the complexity and diversity within a   │
│                   │           │           │        │ specific group or population                               │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=650030;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15890\Neuronpedia 15890]8;;\ │ 0.2600    │ 0.0008    │ 325.00 │  punctuation and formatting elements in the text           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=1427;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/604\Neuronpedia 604]8;;\   │ 0.1200    │ 0.0004    │ 300.00 │  expressions indicating capability or possibility          │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=982091;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4580\Neuronpedia 4580]8;;\  │ 0.1400    │ 0.0005    │ 280.00 │  phrases that indicate criticism or disdain for political  │
│                   │           │           │        │ or social issues                                           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=97409;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10959\Neuronpedia 10959]8;;\ │ 0.2600    │ 0.0012    │ 216.67 │  punctuation and sentence structures in written language   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=540922;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/9018\Neuronpedia 9018]8;;\  │ 0.2600    │ 0.0030    │ 86.67  │  phrases related to guidelines and support systems         │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=762305;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/12020\Neuronpedia 12020]8;;\ │ 0.1200    │ 0.0022    │ 54.55  │  references to manipulative or deceitful behaviors in      │
│                   │           │           │        │ relation to political or economic contexts                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=705445;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10464\Neuronpedia 10464]8;;\ │ 0.1800    │ 0.0034    │ 52.94  │ mentions of deceitful or fraudulent activities and         │
│                   │           │           │        │ concepts                                                   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=935105;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/13809\Neuronpedia 13809]8;;\ │ 0.2200    

        Interesting latents more common in strongreject's "Hate, harassment and discrimination" than alpaca        
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=70015;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4580\Neuronpedia 4580]8;;\  │ 0.1400    │ 0.0005    │ 280.00 │  phrases that indicate criticism or disdain for political  │
│                   │           │           │        │ or social issues                                           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=554901;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.2400    │ 0.0009    │ 266.67 │ various sentence structures and punctuation                │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=974556;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/2363\Neuronpedia 2363]8;;\  │ 0.2600    │ 0.0022    │ 118.18 │ terms associated with sexual violence and abuse            │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=395795;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4880\Neuronpedia 4880]8;;\  │ 0.2800    │ 0.0036    │ 77.78  │ terms related to racial issues and discrimination          │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=722446;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/7921\Neuronpedia 7921]8;;\  │ 0.1600    │ 0.0025    │ 64.00  │ words and phrases related to LGBTQ+ identities and issues  │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=613139;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/12020\Neuronpedia 12020]8;;\ │ 0.1400    │ 0.0022    │ 63.64  │  references to manipulative or deceitful behaviors in      │
│                   │           │           │        │ relation to political or economic contexts                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=775777;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/11778\Neuronpedia 11778]8;;\ │ 0.1200    │ 0.0029    │ 41.38  │ references to fascism, extremism, and far-right political  │
│                   │           │           │        │ ideology                                                   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=734203;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/14690\Neuronpedia 14690]8;;\ │ 0.3000    │ 0.0080    │ 37.50  │ terms related to defamatory statements and accusations     │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=82008;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/5502\Neuronpedia 5502]8;;\  │ 0.1400    │ 0.0043    │ 32.56  │ references to organized crime and illegal activities,      │
│                   │           │           │        │ particularly those involving drug trafficking              │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=240067;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4950\Neuronpedia 4950]8;;\  │ 0.1600    │

            Interesting latents more common in strongreject's "Illegal goods and services" than alpaca             
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio   ┃ Explanation                                               ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=477799;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10042\Neuronpedia 10042]8;;\ │ 0.1200    │ 0.0001    │ 1200.00 │ phrases and words related to educational resources and    │
│                   │           │           │         │ ideas                                                     │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=977576;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/11135\Neuronpedia 11135]8;;\ │ 0.1400    │ 0.0003    │ 466.67  │ expressions of certainty or affirmation                   │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=265149;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15275\Neuronpedia 15275]8;;\ │ 0.1800    │ 0.0004    │ 450.00  │ references to instructions and guidelines                 │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=65172;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.2200    │ 0.0009    │ 244.44  │ various sentence structures and punctuation               │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=596442;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/13708\Neuronpedia 13708]8;;\ │ 0.1200    │ 0.0005    │ 240.00  │ conversational phrases and terms indicating agreement or  │
│                   │           │           │         │ affirmation                                               │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=173416;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/7866\Neuronpedia 7866]8;;\  │ 0.1200    │ 0.0006    │ 200.00  │ descriptions of items or services that include specific   │
│                   │           │           │         │ features and packaging details                            │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=345329;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10024\Neuronpedia 10024]8;;\ │ 0.1200    │ 0.0006    │ 200.00  │ phrases indicating thorough or extensive evaluations and  │
│                   │           │           │         │ analyses                                                  │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=845916;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1668\Neuronpedia 1668]8;;\  │ 0.2000    │ 0.0015    │ 133.33  │  phrases and terms related to detailed descriptions and   │
│                   │           │           │         │ analyses                                                  │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id=169204;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/13587\Neuronpedia 13587]8;;\ │ 0.1600    │ 0.0012    │ 133.33  │ references to ownership or possession                     │
├───────────────────┼───────────┼───────────┼─────────┼───────────────────────────────────────────────────────────┤
│ ]8;id

                Interesting latents more common in strongreject's "Non-violent crimes" than alpaca                 
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=247458;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/3662\Neuronpedia 3662]8;;\  │ 0.1356    │ 0.0004    │ 338.98 │ statements of existence or non-existence along with        │
│                   │           │           │        │ related verbs like "is" and "are"                          │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=836431;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15890\Neuronpedia 15890]8;;\ │ 0.1525    │ 0.0008    │ 190.68 │  punctuation and formatting elements in the text           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=876234;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.1525    │ 0.0009    │ 169.49 │ various sentence structures and punctuation                │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=54304;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1539\Neuronpedia 1539]8;;\  │ 0.1356    │ 0.0009    │ 150.66 │  gerunds and their association with actions or states      │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=920451;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/4153\Neuronpedia 4153]8;;\  │ 0.1017    │ 0.0009    │ 112.99 │  phrases related to access to various resources and        │
│                   │           │           │        │ services                                                   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=319491;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/5502\Neuronpedia 5502]8;;\  │ 0.3390    │ 0.0043    │ 78.83  │ references to organized crime and illegal activities,      │
│                   │           │           │        │ particularly those involving drug trafficking              │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=21523;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15881\Neuronpedia 15881]8;;\ │ 0.1017    │ 0.0016    │ 63.56  │  phrases related to theft and stealing                     │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=705066;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15224\Neuronpedia 15224]8;;\ │ 0.1017    │ 0.0016    │ 63.56  │  references to legal relationships and agreements related  │
│                   │           │           │        │ to trusts                                                  │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=569895;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15163\Neuronpedia 15163]8;;\ │ 0.1525    │ 0.0024    │ 63.56  │  references to investigations and surveillance related to  │
│                   │           │           │        │ political figures and entities                             │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=34

                  Interesting latents more common in strongreject's "Sexual content" than alpaca                   
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=790202;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.2600    │ 0.0009    │ 288.89 │ various sentence structures and punctuation                │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=778495;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10959\Neuronpedia 10959]8;;\ │ 0.2400    │ 0.0012    │ 200.00 │  punctuation and sentence structures in written language   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=37987;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1539\Neuronpedia 1539]8;;\  │ 0.1600    │ 0.0009    │ 177.78 │  gerunds and their association with actions or states      │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=212554;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15890\Neuronpedia 15890]8;;\ │ 0.1400    │ 0.0008    │ 175.00 │  punctuation and formatting elements in the text           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=743132;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/2363\Neuronpedia 2363]8;;\  │ 0.3000    │ 0.0022    │ 136.36 │ terms associated with sexual violence and abuse            │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=503787;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/5924\Neuronpedia 5924]8;;\  │ 0.1400    │ 0.0011    │ 127.27 │ phrases indicating maintenance and management of quality   │
│                   │           │           │        │ or stability                                               │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=638260;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15163\Neuronpedia 15163]8;;\ │ 0.2800    │ 0.0024    │ 116.67 │  references to investigations and surveillance related to  │
│                   │           │           │        │ political figures and entities                             │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=873326;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/6869\Neuronpedia 6869]8;;\  │ 0.1400    │ 0.0014    │ 100.00 │ phrases about conflicts of interest and ethical            │
│                   │           │           │        │ disclosures in research                                    │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=249468;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/8513\Neuronpedia 8513]8;;\  │ 0.1800    │ 0.0024    │ 75.00  │ terms and phrases related to public acknowledgment and     │
│                   │           │           │        │ revelation                                                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=741046;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15902\Neuronpedia 15902]8;;\ │ 0.1400   

                     Interesting latents more common in strongreject's "Violence" than alpaca                      
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Latent            ┃ Frac harm ┃ Frac base ┃ Ratio  ┃ Explanation                                                ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ]8;id=140826;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/1755\Neuronpedia 1755]8;;\  │ 0.2778    │ 0.0009    │ 308.64 │ various sentence structures and punctuation                │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=81118;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/15890\Neuronpedia 15890]8;;\ │ 0.1481    │ 0.0008    │ 185.19 │  punctuation and formatting elements in the text           │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=391042;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/10959\Neuronpedia 10959]8;;\ │ 0.2037    │ 0.0012    │ 169.75 │  punctuation and sentence structures in written language   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=679783;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/11970\Neuronpedia 11970]8;;\ │ 0.1481    │ 0.0016    │ 92.59  │ military terminology related to aerospace and defense      │
│                   │           │           │        │ operations                                                 │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=867533;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/13809\Neuronpedia 13809]8;;\ │ 0.2407    │ 0.0047    │ 51.22  │ requests for assistance or suggestions on various topics   │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=560689;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/786\Neuronpedia 786]8;;\   │ 0.1111    │ 0.0030    │ 37.04  │ specific behaviors deemed unacceptable in a professional   │
│                   │           │           │        │ setting                                                    │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=485681;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/3647\Neuronpedia 3647]8;;\  │ 0.1296    │ 0.0048    │ 27.01  │  phrases and words related to computational processes and  │
│                   │           │           │        │ decision-making                                            │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=366836;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/3507\Neuronpedia 3507]8;;\  │ 0.1111    │ 0.0054    │ 20.58  │ probabilities and their related calculations in the        │
│                   │           │           │        │ context of statistics                                      │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=861390;https://www.neuronpedia.org/feature/gemma-2-2b/5-gemmascope-res-16k/14175\Neuronpedia 14175]8;;\ │ 0.1852    │ 0.0097    │ 19.09  │  phrases related to events, particularly situations        │
│                   │           │           │        │ involving potential incidents or emergencies               │
├───────────────────┼───────────┼───────────┼────────┼────────────────────────────────────────────────────────────┤
│ ]8;id=5